In [ ]:
from datetime import datetime #날짜와 시간을 쉽게 조작할 수 있게 하는 클래스 제공
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
from pathlib import Path

folder = "bigcon2020/data"
base_path = Path("/content/drive/My Drive/")
project_path = base_path / folder
os.chdir(project_path)
for x in list(project_path.glob("*")):
    if x.is_dir():
        dir_name = str(x.relative_to(project_path))
        os.rename(dir_name, dir_name.split(" ", 1)[0])
print(f"현재 디렉토리 위치: {os.getcwd()}")

현재 디렉토리 위치: /content/drive/My Drive/bigcon2020/data


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

print('tensorflow version: {}'.format(tf.__version__))
print('GPU 사용 가능 여부: {}'.format(tf.test.is_gpu_available()))

tensorflow version: 2.3.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU 사용 가능 여부: True


In [ ]:
batch_size = 9
max_epochs = 30
learning_rate = 3e-5
hidden_sizes = [100, 100]

In [ ]:
def preprocess():
  # 1차_최종_공통 : 시즌, 홈어웨이, 경기간간격, 득점, 실점, 승패
  general = pd.read_csv('1차_최종_공통.csv')
  general = general[general['팀코드']=='NC']
  general = general[['게임키', '경기간간격','득점','실점','결과']]
  general['시즌'] = general['게임키'].apply(lambda x:int(x[:4]))
  general['홈어웨이'] = general['게임키'].apply(lambda x:0 if x[-3:-1]=='NC' else 1)
  general=general.set_index('게임키')
  # 팀타자 : 기준팀 wRC
  hitter = pd.read_csv('./1차_최종_팀타자.csv')
  hitter_vs = hitter[hitter['상대팀코드']=='NC']
  hitter = hitter[hitter['팀코드']=='NC']
  hitter = hitter[['게임키', 'wRC']]
  hitter_vs = hitter_vs[['게임키', 'wRC']]
  hitter=hitter.set_index('게임키')
  hitter_vs=hitter_vs.set_index('게임키')
  hitter_vs=hitter_vs.add_prefix('vs_')
  # 팀투수 : 기준팀, 상대팀 RA/9, whip
  pitcher = pd.read_csv('./1차_최종_팀투수.csv')
  pitcher_vs = pitcher[pitcher['상대팀코드']=='NC']
  pitcher = pitcher[pitcher['팀코드']=='NC']
  pitcher = pitcher[['게임키', 'RA/9','whip']]
  pitcher_vs = pitcher_vs[['게임키', 'RA/9','whip']]
  pitcher = pitcher.set_index('게임키')
  pitcher_vs = pitcher_vs.set_index('게임키')
  pitcher_vs = pitcher_vs.add_prefix('vs_')
  df = pd.concat([general, hitter,hitter_vs, pitcher,pitcher_vs], axis=1)
  df= df[['시즌','경기간간격','홈어웨이','wRC','vs_wRC','RA/9','whip','vs_RA/9','vs_whip','득점','실점','결과']]
  df['결과'] = df['결과'].apply(lambda x : 0 if x =='W' else 1)
  return df

In [ ]:
df = preprocess()
df.head()

NameError: ignored

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 639 entries, 20160401HTNC0 to 20200719KTNC0
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   시즌       639 non-null    int64  
 1   경기간간격    639 non-null    float64
 2   홈어웨이     639 non-null    int64  
 3   wRC      639 non-null    float64
 4   vs_wRC   639 non-null    float64
 5   RA/9     639 non-null    float64
 6   whip     639 non-null    float64
 7   vs_RA/9  639 non-null    float64
 8   vs_whip  639 non-null    float64
 9   득점       639 non-null    int64  
 10  실점       639 non-null    int64  
 11  결과       639 non-null    object 
dtypes: float64(7), int64(4), object(1)
memory usage: 64.9+ KB


In [ ]:
train_data= np.asarray(df_z[(df_z['시즌']!= 2019) & (df_z['시즌']!= 2020)].drop('시즌', axis=1)).astype(np.float32)
val_data= np.asarray(df_z[df_z['시즌']==2019].drop('시즌', axis=1)).astype(np.float32)
test_data= np.asarray(df_z[df_z['시즌']==2020].drop('시즌', axis=1)).astype(np.float32)

In [ ]:
print("shape of train data: {}".format(train_data.shape))
print("shape of val data: {}".format(val_data.shape))
print("shape of test data: {}".format(test_data.shape))

shape of train data: (432, 12)
shape of val data: (144, 12)
shape of test data: (63, 12)


In [ ]:
def make_dataset(data, seq_length=5, target_delay=0, strides=1):
    sequence = []
    target = []
    for index in range(len(data) - seq_length - target_delay):
        if index % strides == 0:
          sequence.append(data[index:index+seq_length])
          target.append(data[index+seq_length+target_delay][-1])
    return np.array(sequence), np.array(target)

In [ ]:
train_sequences, train_labels = make_dataset(train_data)
val_sequences, val_labels = make_dataset(val_data)
test_sequences, test_labels = make_dataset(test_data)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_sequences, train_labels))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.repeat()

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_sequences, test_labels))
test_dataset = test_dataset.batch(batch_size)
print(test_dataset)
print(test_labels)

<BatchDataset shapes: ((None, 5, 12), (None,)), types: (tf.float32, tf.float32)>
[0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 1. 1. 1. 0. 1. 0. 0.]


In [ ]:
def eval_baseline(dataset, loss_fn):
    mean_loss = tf.keras.metrics.Mean()
    for sequences, targets in dataset:
        predictions = sequences[:,-1,0]
        loss = loss_fn(predictions, targets)
        mean_loss(loss)
        
    print('Baseline Average Loss: {:.4f}'.format(mean_loss.result()))
    return mean_loss.result()

In [ ]:
loss_object = tf.keras.losses.MeanSquaredError()
baseline_loss = eval_baseline(test_dataset, loss_object)

Baseline Average Loss: 13.6389


In [ ]:
for i in range(10):
    data_idx = np.random.randint(len(test_sequences))
    pred = test_sequences[data_idx, -1, -1]
    pred = pred
    target = test_labels[data_idx]
    print('예측 경기결과: {:.1f} / 실제 경기결과: {:.1f} / 차이: {:.1f}'.format(
        pred, target, pred - target))

예측 경기결과: 0.0 / 실제 경기결과: 1.0 / 차이: -1.0
예측 경기결과: 1.0 / 실제 경기결과: 0.0 / 차이: 1.0
예측 경기결과: 0.0 / 실제 경기결과: 0.0 / 차이: 0.0
예측 경기결과: 1.0 / 실제 경기결과: 0.0 / 차이: 1.0
예측 경기결과: 0.0 / 실제 경기결과: 1.0 / 차이: -1.0
예측 경기결과: 1.0 / 실제 경기결과: 1.0 / 차이: 0.0
예측 경기결과: 0.0 / 실제 경기결과: 1.0 / 차이: -1.0
예측 경기결과: 0.0 / 실제 경기결과: 0.0 / 차이: 0.0
예측 경기결과: 0.0 / 실제 경기결과: 1.0 / 차이: -1.0
예측 경기결과: 0.0 / 실제 경기결과: 0.0 / 차이: 0.0


In [ ]:
model = tf.keras.Sequential()

In [ ]:
num_layers = len(hidden_sizes)
for i in range(num_layers - 1):
    model.add(tf.keras.layers.LSTM(100,return_sequences=True))
    model.add(tf.keras.layers.LSTM(100,return_sequences=False))
    model.add(tf.keras.layers.Dense(1))

In [ ]:
for step, (sequences, targets) in enumerate(train_dataset.take(1)):
    predictions = model(sequences)

In [ ]:
model.compile(loss="MSE", 
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=["MSE"])

In [ ]:
checkpoint_path = './checkpoint/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                             save_best_only=True,  
                             verbose=1
                            )

In [ ]:
model.fit(train_dataset, steps_per_epoch = train_data.shape[1]/batch_size, 
          validation_data = val_dataset, 
          validation_steps= val_data.shape[1]/batch_size,
          epochs= max_epochs,
          callbacks= [checkpoint])

Epoch 1/30
1/1 [=====================>........] - ETA: 0s - loss: 0.2276 - MSE: 0.2276
Epoch 00001: val_loss did not improve from 0.25534
2/1 [=============================================] - 0s 11ms/step - loss: 0.2394 - MSE: 0.2394 - val_loss: 0.3931 - val_MSE: 0.3931
Epoch 2/30
1/1 [=====================>........] - ETA: 0s - loss: 0.2231 - MSE: 0.2231
Epoch 00002: val_loss did not improve from 0.25534
2/1 [=============================================] - 0s 8ms/step - loss: 0.2972 - MSE: 0.2972 - val_loss: 0.3077 - val_MSE: 0.3077
Epoch 3/30
1/1 [=====================>........] - ETA: 0s - loss: 0.1804 - MSE: 0.1804
Epoch 00003: val_loss did not improve from 0.25534
2/1 [=============================================] - 0s 8ms/step - loss: 0.1583 - MSE: 0.1583 - val_loss: 0.2807 - val_MSE: 0.2807
Epoch 4/30
1/1 [=====================>........] - ETA: 0s - loss: 0.3247 - MSE: 0.3247
Epoch 00004: val_loss did not improve from 0.25534
2/1 [=============================================]

In [ ]:
loss, acc = model.evaluate(test_sequences,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

2/2 - 0s - loss: 0.2421 - MSE: 0.2421
복원된 모델의 정확도: 24.21%


In [ ]:
model = tf.keras.Sequential()
num_layers = len(hidden_sizes)

for i in range(num_layers - 1):
    model.add(tf.keras.layers.LSTM(hidden_sizes[0],return_sequences=True))
    model.add(tf.keras.layers.LSTM(hidden_sizes[-1],return_sequences=False))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss="MSE", 
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=["MSE"])

In [ ]:
loss, acc = model.evaluate(test_sequences,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

2/2 - 0s - loss: 0.6067 - MSE: 0.6067
복원된 모델의 정확도: 60.67%


In [ ]:
for sequences, targets in train_dataset.take(1):
    outputs = model(sequences)
    print(outputs)

tf.Tensor(
[[-0.09230894]
 [-0.09701811]
 [-0.10126693]
 [-0.09724387]
 [-0.12011196]
 [-0.11684848]
 [-0.10861947]
 [-0.11267105]
 [-0.11559678]], shape=(9, 1), dtype=float32)


In [ ]:
model.load_weights(checkpoint_dir)

In [ ]:
_, test_loss = model.evaluate(test_dataset, steps=len(test_sequences)/batch_size)

7/6 [================================] - 0s 2ms/step - loss: 0.3551 - MSE: 0.3551


In [ ]:
for i in range(10):
    data_idx = np.random.randint(len(test_sequences))
    pred = test_sequences[data_idx, -1, -1]
    pred = pred
    target = test_labels[data_idx]
    print('예측 경기결과: {:.1f} / 실제 경기결과: {:.1f} / 차이: {:.1f}'.format(
        pred, target, pred - target))

예측 경기결과: 0.0 / 실제 경기결과: 1.0 / 차이: -1.0
예측 경기결과: 0.0 / 실제 경기결과: 0.0 / 차이: 0.0
예측 경기결과: 0.0 / 실제 경기결과: 1.0 / 차이: -1.0
예측 경기결과: 1.0 / 실제 경기결과: 0.0 / 차이: 1.0
예측 경기결과: 0.0 / 실제 경기결과: 1.0 / 차이: -1.0
예측 경기결과: 1.0 / 실제 경기결과: 0.0 / 차이: 1.0
예측 경기결과: 1.0 / 실제 경기결과: 1.0 / 차이: 0.0
예측 경기결과: 1.0 / 실제 경기결과: 0.0 / 차이: 1.0
예측 경기결과: 0.0 / 실제 경기결과: 1.0 / 차이: -1.0
예측 경기결과: 0.0 / 실제 경기결과: 0.0 / 차이: 0.0


In [ ]:
loss, acc = model.evaluate(test_sequences,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

2/2 - 0s - loss: 0.3551 - MSE: 0.3551
복원된 모델의 정확도: 35.51%
